# Artificial Neural Network for prediciting Camber and Toe extrema

## Install any missing libraries

In [ ]:
!pip3 install openpyxl

## Import all libraries

In [1]:
import glob 
import random
import numpy as np
import pandas as pd
import math
import os
import copy
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras import layers
from keras.optimizers import SGD, Adam

## Importing, Cleaning, and Preparing Data

In [2]:
files=glob.glob("data/*.xlsx",recursive=True)
population=[]
for i in range(0, len(files)):
    parent=pd.read_excel(files[i],header=None).values
    population.append(parent)
population=np.array(population,dtype=object)

In [3]:
all_geometry = []
all_results = []
for i in range(0, len(population)):
    pop = pd.DataFrame(population[i])
    pop = pop.drop(index=[20, 21], axis=0)
    geometry = pop.iloc[14:22, 1:4].values
    results = pop.iloc[82:96, 1:6].values
    all_geometry.append(geometry)
    all_results.append(results)
all_geometry = np.array(all_geometry)
all_results = np.array(all_results)

In [4]:
X_flat=all_geometry.reshape((all_geometry.shape[0], -1))

In [5]:
X=[]
output=[]
for i in range(X_flat.shape[0]):
    for j in range(all_results[i].shape[0]):
        X.append(list(np.append(X_flat[i],all_results[i][j][0])))
        output.append(list(all_results[i][j][1:]))
X=np.array(X)
output=np.array(output)

In [6]:
#output=np.empty(shape=[all_results.shape[0],4])
#for i in range(all_results.shape[0]):
    #output[i,0]=max(all_results[i,:,0])
    #output[i,1]=min(all_results[i,:,0])
    #output[i,2]=max(all_results[i,:,1])
    #output[i,3]=min(all_results[i,:,1])

In [10]:
y=output
X=X.astype('float32')
y=y.astype('float32')

#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

train_df=pd.DataFrame(X)
train_df[25]=y[:,0]
train_df[26]=y[:,1]
train_df[27]=y[:,2]
train_df[28]=y[:,3]
target=train_df[24:28]
idx=[]
for i in range(train_df.shape[0]):
    if (train_df[26][i]>2 or train_df[26][i]<-2) :
        idx.append(i)
train_df = train_df.drop(index = idx, axis=0)
train_df.shape
train_df.to_csv('toe_optimized_data.csv')

## Scaling Data between 0 and 1

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_train = scaler.fit_transform(train_df)

# Print out the adjustment that the scaler applied to the data
print("Note: Median values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[28], scaler.min_[28]))
multiplied_by = scaler.scale_[28]
added = scaler.min_[28]

scaled_train_df = pd.DataFrame(scaled_train, columns=train_df.columns.values)
scaled_train_df

## ANN Model

In [ ]:

model = Sequential()
model.add(Dense(25, activation=layers.LeakyReLU()))
model.add(Dense(50, activation=layers.LeakyReLU()))
model.add(Dense(100, activation=layers.LeakyReLU()))
model.add(Dense(100, activation=layers.LeakyReLU()))
model.add(Dense(50, activation=layers.LeakyReLU()))
model.add(Dense(25, activation=layers.LeakyReLU()))
model.add(Dense(4,activation=layers.LeakyReLU()))
adam = Adam(lr=0.005)
model.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy'])

In [ ]:
X = scaled_train_df[scaled_train_df.columns[:25]].values
Y = scaled_train_df[scaled_train_df.columns[25:29]].values

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=4000,
    batch_size=10,
    shuffle=True,
    verbose=1
)

## Predictions

In [ ]:
prediction = model.predict(X_train[:5])
print('Prediction with scaling - ',format(prediction))

y_pred = prediction
y_pred -= added
y_pred /= multiplied_by
print("Prediction - ",format(y_pred))

In [ ]:
y_train[:5]

In [ ]:
Y_pred=model.predict(X_test)

In [ ]:
data_pred=np.concatenate((X_test,y_test),axis=1)
data_test=np.concatenate((X_test,Y_pred),axis=1)

In [ ]:
data_pred_scaled=scaler.inverse_transform(data_pred)
data_test_scaled=scaler.inverse_transform(data_test)

In [ ]:
from matplotlib import pyplot as plt
y_test_scaled=data_test_scaled[:,25:]
y_pred_scaled=data_pred_scaled[:,25:]
y_ac=y_test_scaled-y_pred_scaled
plt.plot(y_ac[:,1])